In [8]:
import pandas as pd
import numpy as np
import pickle 
import os

In [9]:

# Step 1: Open the file in read-binary mode
with open('data.pkl', 'rb') as file:
    # Step 2: Load the data from the file
    data = pickle.load(file)

# Now, `data` contains the deserialized Python object
print(data)


0        JAN JAGRAN TIMES
1        JAGRAN CITY PLUS
2         SAMPURNA JAGRAN
3           DAINIK JAGRAN
4           VISHWA JAGRAN
               ...       
21394        KAIWART AWAZ
21395     SARBAHARAR AWAZ
21396      SHRAMIKER AWAZ
21397          SOBAR AWAZ
21398        AWAZ AAP TAK
Name: Title Name, Length: 10790, dtype: object


In [10]:
data = data.str.lower()
print(data)

0        jan jagran times
1        jagran city plus
2         sampurna jagran
3           dainik jagran
4           vishwa jagran
               ...       
21394        kaiwart awaz
21395     sarbaharar awaz
21396      shramiker awaz
21397          sobar awaz
21398        awaz aap tak
Name: Title Name, Length: 10790, dtype: object


In [11]:
data = pd.DataFrame(data)
data = data.drop_duplicates().dropna()

In [12]:
data.shape

(10790, 1)

In [13]:
data.rename(columns={'Title Name': 'title'}, inplace=True)


In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document  # Import Document class

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Assuming your DataFrame `data` has a column 'title' containing the titles or documents
# Create a list of Document objects
documents = [Document(page_content=row['title']) for index, row in data.iterrows()]

# Split the documents
split_docs = text_splitter.split_documents(documents)

In [15]:
from langchain_ollama.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model ="mxbai-embed-large")

In [4]:
from dotenv import load_dotenv
load_dotenv()
import os
api_key = os.getenv("PINECONE_API_KEY")

In [5]:
from pinecone import Pinecone, ServerlessSpec
index_name = "sliftex"
pc = Pinecone(api_key=api_key)
index = pc.Index(index_name)

In [86]:
import nltk
nltk.download('punkt', download_dir='./nltk_data')


[nltk_data] Downloading package punkt to ./nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [96]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Debanjan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [16]:
import nltk
from pinecone_text.sparse import BM25Encoder

# Download the necessary tokenizer data
nltk.download('punkt')
nltk.data.path.append(r'C:\Users\Debanjan\AppData\Roaming\nltk_data')

# Initialize the BM25 encoder
encoder = BM25Encoder().default()

# Your data and BM25 encoding process
# Ensure the data column is converted to a list of strings
corpus = data['title'].tolist()  # Convert the column to a list

# Fit the encoder
encoder.fit(corpus)

# Save the encoded data
encoder.dump("document.json")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Debanjan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 10790/10790 [00:00<00:00, 16277.66it/s]


In [6]:
encoded_docs = BM25Encoder().load("document.json")

In [17]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
retriever = PineconeHybridSearchRetriever(index=index, sparse_encoder=encoded_docs, embeddings=embeddings)

In [18]:
retriever

PineconeHybridSearchRetriever(embeddings=OllamaEmbeddings(model='mxbai-embed-large', base_url=None, client_kwargs={}), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x00000204AB81D0A0>, index=<pinecone.data.index.Index object at 0x00000204D84929C0>)

In [105]:
retriever.add_texts(
    corpus
)

100%|██████████| 338/338 [07:21<00:00,  1.31s/it]


In [19]:
retriever.invoke("Chutiya")

[Document(metadata={'score': 0.343915164}, page_content='pudhiya india'),
 Document(metadata={'score': 0.312052101}, page_content='pratigya times'),
 Document(metadata={'score': 0.308253974}, page_content='india yug'),
 Document(metadata={'score': 0.304049164}, page_content='charaniya times')]